# Lab 2.2: Work distribution

Once we have created the workers (i.e. teams and threads as seen in [Lab2.1](lab2.1.ipynb)), the question is how do we assign work to each of the threads. We have already seen some examples of work assignment. In this lab we will explore how can we further control the work that gets assigned to the workers.

## Table of content

1. Parallel regions
2. Controlling threadas
    1. Masked directive
    2. Single directive
    3. Critical directive
    4. Sections directive
    5. Barriers
3. Worksharing loops
    2. For directive
4. Loop scheduling
    1. Static scheduling
    2. Dynamic scheduling
    3. Guided scheduling
    4. Runtime scheduling
    5. Auto scheduling

## Parallel regions

As seen previously, the `parallel`` directive allows to generate threads. However, parallel directives have an associated region of code that corresponds to the work that each thread is to perform. This is, the enclosed region of code that is associated with the parallel region. 

```C
int i = 0;
#pragma omp parallel
{
    // This is the work associated to each thread
    #pragma omp atomic 
    i = i + 1;
}
```


In [1]:
#build
!srun -N 1 -c 8 clang -fopenmp C/simple_parallel.c -o C/simple_parallel.exe

In [2]:
#execute
!srun -N 1 -c 8 C/./simple_parallel.exe

i = 4


To play with this code go to [simple_parallel.c](C/simple_parallel.c)

Since parallel regions use the fork-joint model, each thread is assigned the same functionality. Therefore, the first work assigment we do in parallel regions is equivalent to "every thread executes the same region".

In many cases we want to be able to change this behavior and provide more control over work assignment. The rest of this section will focus on this.

It is possible to use the thread ID to identify the different threads and assign different functionalities. Take for example the following code (`NOTE: This code is not recommended and is somewhat a bad practice`:)

```C
#pragma omp parallel num_threads(5)
{
    if(omp_get_thread_num() == 0) {
        printf("Hi, I am thread 0\n");
    }
    if(omp_get_thread_num() == 4) {
        printf("And I am thread 4\n");
    }
}
```

In [4]:
#build
!srun -N 1 -c 8 clang -fopenmp C/not_recommended_parallel_if.c -o C/not_recommended_parallel_if.exe

In [5]:
#execute
!srun -N 1 -c 8 C/./not_recommended_parallel_if.exe

Hi, I am thread 0
And I am thread 4


However, there is a better way to control work assignment for threads as we will se below.

To play with the code above go to [not_recommended_parallel_if.c](C/not_recommended_parallel_if.c)

### Masked directive
The `masked` directive (formerly known as `master` but now featuring additional functionality) allows to select a single thread for execution of a region. This directive automatically performs the thread ID check for us, allowing to write segment of code to be associated with a single thread. 

The `filter()` clause can be used to control the threadID. By default the threadID chosen by the masked directive is threadID == 0. The same code above can be re-written as:

```C
#pragma omp parallel num_threads(5)
{
    #pragma omp masked
        printf("Hi, I am thread 0\n");
    #pragma omp masked filter(4)
        printf("And I am thread 4\n");
}
```

Notice that the first `masked` does not uses the `filter` directive, resulting in the equivalent `if(omp_get_thread_num() == 0)`  

In [8]:
#build
!srun -N 1 -c 8 clang -fopenmp C/parallel_masked.c -o C/parallel_masked.exe

In [9]:
#execute
!srun -N 1 -c 8 C/./parallel_masked.exe

Hi, I am thread 0
And I am thread 4


To play with this code go to [parallel_masked.c](C/parallel_masked.c)

### Single directive
The mask directive allows us to specify a single thread, however, what if we do not really care which thread executes the region, as long as it is only one. The `single` directive has a similar effect to `masked`, but instead of selecting the thread, any thread may be in charge of executing the enclosed region. Take for example the following code:

```C
#pragma omp parallel num_threads(10)
{
    #pragma omp single
        printf("1, I am thread %d\n", omp_get_thread_num());
    #pragma omp single
        printf("2, I am thread %d\n", omp_get_thread_num());
    #pragma omp single
        printf("3, I am thread %d\n", omp_get_thread_num());
} 
```

In this example, the three regins will be executed in parallel. Each region will be executed only once. And the thread that executes the region can be any among the 10 threads

In [10]:
#build
!srun -N 1 -c 8 clang -fopenmp C/parallel_single.c -o C/parallel_single.exe

In [13]:
#execute
!srun -N 1 -c 8 C/./parallel_single.exe

1, I am thread 3
2, I am thread 0
3, I am thread 4


Run the example code multiple times, and notice if the thread that executes the region changes every time. 

To play with this code open the fine [parallel_single.c](C/parallel_single.c)

Another difference with the masked directive is that single has an implicit barrier at the end of the region. Meaning, all threads must synchronize before continuing the execution of the region after the single.

```C
int i = 0;
#pragma omp parallel
{
    #pragma omp single
        i = omp_get_num_threads();
    #pragma omp atomic
        i = i - 1;
}
printf("i = %d\n", i);
```

In [39]:
#build
!srun -N 1 -c 8 clang -fopenmp C/single_barrier.c -o C/single_barrier.exe

In [40]:
#execute
!srun -N 1 -c 8 C/./single_barrier.exe

i = 0


The code above works well because before performing the atomic operation, all threads must wait for the single operaiton to finish executing.

You can play with this code in [single_barrier.c](C/single_barrier.c)

It is possible to remove the implicit barrier introduced by the `single` directive by using the `nowait` clause. This will allow for the other threads to execute in parallel with the single region, while still allowing only a single thread to execute the single region.

### Exercise 1 - Discussion question
Can you explain why the following code not always returns 0?

```C
int i = 0;
#pragma omp parallel
{
    #pragma omp single nowait
        i = omp_get_num_threads();
    #pragma omp atomic
        i = i - 1;
}
printf("i = %d\n", i);
```

In [50]:
#build
!srun -N 1 -c 8 clang -fopenmp Exercises/single_no_barrier.c -o Exercises/single_no_barrier.exe

In [57]:
#execute 10 times
!for i in `seq 10`; do srun -N 1 -c 8 Exercises/./single_no_barrier.exe; done

i = 0
i = 1
i = 0
i = 0
i = 0
i = 18
i = 0
i = 2
i = 3
i = 0


You can play with this code in [Exercises/single_no_barrier.c](Exercises/single_no_barrier.c)

### Critical directive
Neither the `single` or the `masked` directives are mutually exclussive between threads. This means, that while a single or masked region executes, it is possible for other threads to perform operations. 

Critical sections are sections that have a guaranteed mutual exclusion. Mutual exclusion is an important property of parallel programming since it protects access to sections of the code that are not thread safe. Unlike atomic operations that can only affect a single memory region, mutual exclusion allows to guard a section of code, rather than a single memory location. 

Let's take the following **broken** code as an example.

```C
int i, j[10] = {0, 0, 0, 0, 0, 0, 0, 0, 0, 0};

#pragma omp parallel
{
    i = omp_get_thread_num();
    j[i] = 99;
}

for (i = 0; i < 10; i++)
    printf("j[%d] = %d\n", i, j[i]);
```

In [71]:
#build
!srun -N 1 -c 8 clang -fopenmp C/broken_code_no_critical.c -o C/broken_code_no_critical.exe

In [72]:
#execute
!srun -N 1 -c 8 C/./broken_code_no_critical.exe

j[0] = 99
j[1] = 99
j[2] = 99
j[3] = 99
j[4] = 99
j[5] = 99
j[6] = 0
j[7] = 0
j[8] = 0
j[9] = 0


This code clearly has a race condition on i, resulting in an incorrect memory access on j.

Can we fix this by using atomic operations? Give it a try in [C/broken_code_no_critica.c](C/broken_code_no_critica.c)

The answer is no. The reason is that there is a set of operations that spawn beyond a single memory access that need to occur exclussively from one another. Imagine the following execution order of the code above:

```
T1: i = 1;
T2: i = 2;
T1: j[i] = 99;
T2: j[i] = 99;

```

In this case, T2 will have the right result, but the fact that i was modified before T1 was able to use the result of `omp_get_thread_num()` implies that `j[1] = 99` cannot occur. 

In general one must be careful about operations that need to happen exclusively. Whenever the oepration goes beyond a simple atomic memory access, it is necessary to use critical regions.

The `critical` directive allows us to do exactly this. We use `critical` to select a large number of operations that are known not to be thread safe (e.g., using a function that is not thread safe), to guarantee that only a single thread executes such function.

Critical does not reduce the number of executions in the region, but just guarantees that each execution is independent from each other. Take for example the following code that uses an array to store the order in which threads have executed the critical region.

```C
int i = 0, j[10] = {0, 0, 0, 0, 0, 0, 0, 0, 0, 0};

#pragma omp parallel
{
    #pragma omp critical
    {
        i++;
        j[i] = omp_get_thread_num();
        #pragma omp flush
    }
}

for (i = 0; i < 10; i++)
    printf("j[%d] = %d\n", i, j[i]);
```

In [77]:
#build
!srun -N 1 -c 8 clang -fopenmp C/critical_array_order.c -o C/critical_array_order.exe

In [78]:
#execute
!srun -N 1 -c 8 C/./critical_array_order.exe

Thread 0 was the 1th to execute the critical section.
Thread 4 was the 2th to execute the critical section.
Thread 3 was the 3th to execute the critical section.
Thread 0 was the 4th to execute the critical section.
Thread 2 was the 5th to execute the critical section.
Thread 1 was the 6th to execute the critical section.
Thread 5 was the 7th to execute the critical section.
Thread 0 was the 8th to execute the critical section.
Thread 0 was the 9th to execute the critical section.
Thread 0 was the 10th to execute the critical section.


You can play with this code in [C/critical_array_order.c](C/critical_array_order.c).

```
NOTE: The #pragma omp flush is necessary because i may be stored in a register in a given thread. We must guarantee that the register is flushed into memory such that other threads can see the updated value of i. This is, in order to "flush" the effects of the execution of this region to memory, such that other threads can see the new value of i, we must guaranteed that the data has been written back to memory. 
```

### Excercise 2: Critical regions for producer/consumer queue

The following program has a problem. The operation between producer and consumer is not thread safe, and has not been guarded properly. For this reason, it is likely this program never finishes, or gives unexpected results (it could even break during runtime). 

Can you modify the file [Exercises/critical_region_queue.cpp](Exercises/critical_region_queue.cpp) to fix this issue by using critical regions?

In [105]:
#build
!srun -N 1 -c 8 clang++ -fopenmp Exercises/critical_region_queue.cpp -o Exercises/critical_region_queue.exe

In [106]:
#execute
!srun -N 1 -c 8 Exercises/./critical_region_queue.exe

double free or corruption (out)


A possible solution to this excercise can be found in [Solutions/critical_region_queue.cpp](Solutions/critical_region_queue.cpp).

In [108]:
#build
!srun -N 1 -c 8 clang++ -fopenmp Solutions/critical_region_queue.cpp -o Solutions/critical_region_queue.exe

In [111]:
#execute
!srun -N 1 -c 8 Solutions/./critical_region_queue.exe

Consumer took 83 from the queue.
Consumer took 86 from the queue.
Consumer took 77 from the queue.
Consumer took 15 from the queue.
Consumer took 93 from the queue.
Consumer took 35 from the queue.
Consumer took 86 from the queue.
Consumer took 92 from the queue.
Consumer took 49 from the queue.
Consumer took 21 from the queue.
Consumer took 62 from the queue.
Consumer took 27 from the queue.
Consumer took 90 from the queue.
Consumer took 59 from the queue.
Consumer took 63 from the queue.
Consumer took 26 from the queue.
Consumer took 40 from the queue.
Consumer took 26 from the queue.
Consumer took 72 from the queue.
Consumer took 36 from the queue.
Consumer took 11 from the queue.
Consumer took 68 from the queue.
Consumer took 67 from the queue.
Consumer took 29 from the queue.
Consumer took 82 from the queue.
Consumer took 30 from the queue.
Consumer took 62 from the queue.
Consumer took 23 from the queue.
Consumer took 67 from the queue.
Consumer took 35 from the queue.
Consumer t

### Sections directive

Another possibility for work assigment to threads is the use of `sections` directive. This directive allows to specify different worksharing sections that are to be executed by different threads. Each section will be assigned a different thread during runtime, resulting in a collection of threads each with independent behavior. 

Assume the following struct exists:

```C
struct someStruct
{
    int a;
    int b;
    int c;
};
```

Take for example the following code:

```C
    struct someStruct s[10];
    #pragma omp parallel num_threads(5)
    {
        #pragma omp sections
        {
            #pragma omp section
            for (int i = 0; i < 10; i++)
                s[i].a = i;
            #pragma omp section
            for (int i = 0; i < 10; i++)
                s[i].b = i;
            #pragma omp section
            for (int i = 0; i < 10; i++)
                s[i].c = i;
        }
    }

```

Notice how each thread is executing on an independent memory region of the array of structs.

In [113]:
#build
!srun -N 1 -c 8 clang -fopenmp C/sections.c -o C/sections.exe

In [114]:
#execute
!srun -N 1 -c 8 C/./sections.exe

s[0].a = 0
s[0].b = 0
s[0].c = 0
s[1].a = 1
s[1].b = 1
s[1].c = 1
s[2].a = 2
s[2].b = 2
s[2].c = 2
s[3].a = 3
s[3].b = 3
s[3].c = 3
s[4].a = 4
s[4].b = 4
s[4].c = 4
s[5].a = 5
s[5].b = 5
s[5].c = 5
s[6].a = 6
s[6].b = 6
s[6].c = 6
s[7].a = 7
s[7].b = 7
s[7].c = 7
s[8].a = 8
s[8].b = 8
s[8].c = 8
s[9].a = 9
s[9].b = 9
s[9].c = 9


### Barriers

Another important mechanism for threads is the need to synchronize threads. More exactly. to guarantee that all the threads have reached a specific part of the program before it can continue the execution. 

Barriers are quite useful depending on the application. Barriers are useful to guarantee that a result has already been commited before continuing with another part of the parallel program. 

Take the following example

```C
int arr[10];
#pragma omp parallel num_threads(10)
{
    arr[omp_get_thread_num()] = omp_get_thread_num();
    #pragma omp barrier
    #pragma omp single
    {
        for (int i = 0; i < 10; i++)
        {
            printf("arr[%d] = %d\n", i, arr[i]);
        }
    }
}
```

The barrier guarantees that all the threads have written their threadId into `arr`. Allowing the print to have the expected result.


In [118]:
#build
!srun -N 1 -c 8 clang -fopenmp C/barriers.c -o C/barriers.exe

In [119]:
#execute
!srun -N 1 -c 8 C/./barriers.exe

arr[0] = 0
arr[1] = 1
arr[2] = 2
arr[3] = 3
arr[4] = 4
arr[5] = 5
arr[6] = 6
arr[7] = 7
arr[8] = 8
arr[9] = 9


While there are many use cases, a common use is that we can reuse the parallelism that has already been generated. In general, thread creation can be a costly operation in some instances. By reusing the parallel region, we reduce the overhead of the runtime. Let us create a simple benchmark that compares the two schemes

```C

auto start = std::chrono::high_resolution_clock::now();
for (int i = 0; i < 1000; i++)
    for (int j = 0; j < 1000; j++)
        #pragma omp parallel
            f();
auto end = std::chrono::high_resolution_clock::now();
std::chrono::duration<double> diff = end - start;
printf("Time independent parallel regions: %f\n", diff.count());

start = std::chrono::high_resolution_clock::now();
for (int i = 0; i < 1000; i++)
    #pragma omp parallel
        for (int j = 0; j < 1000; j++)
        {
            f();
            #pragma omp barrier
        }
end = std::chrono::high_resolution_clock::now();
diff = end - start;
printf("Time single parallel region with barriers: %f\n", diff.count());

```

In [132]:
#build
!srun -N 1 -c 8 clang++ -fopenmp C/parallel_time_creation.cpp -o C/parallel_time_creation.exe

In [133]:
#execute
!srun -N 1 -c 8 C/./parallel_time_creation.exe

Time independent parallel regions: 0.643148
Time single parallel region with barriers: 0.346148


## Worksharing loops

Loops are really common in programs and are important in the context of worksharing, because it is sometimes possible to distribute the iteration space of a loop across the different worker threads. In Loop worksharing each thread will get a subset of the iteration space of the loop, and it will execute only that given subset. 

Loops can be expressed in many ways (e.g. while, for, do-while, etc), but for loop worksharing, they must be in the canonical form. 

In OpenMP, a canonical loop is a loop that has the following form:

```C
for (init_expr; test_expr; incr_expr) {
    body_stmts;
}
```

where `init_expr` initializes the loop counter, `test_expr` tests the loop counter against a loop limit, `incr_expr` increments the loop counter, and `body_stmts` are the statements executed in each iteration of the loop.

It's worth noting that not all loops can be expressed in the canonical form. For example, loops with multiple loop counters in the same loop structure, or loops with non-constant loop limits cannot be expressed in the canonical form and may require additional work to parallelize.

### The for directive

The for directive is used to distribute a loop in a parallel region. Take a look at the following example:

```C
#pragma omp parallel
    #pragma omp for
    for (int i = 0; i < 10; i++)
    {
        printf("Hi from thread %d\n, I am iteration %d\n", omp_get_thread_num(), i);
    }
```

The 10 iterations will be split across the different threads created by the parallel directive.

In [138]:
#build
!srun -N 1 -c 8 clang -fopenmp C/for_loop.c -o C/for_loop.exe

In [140]:
#execute
!srun -N 1 -c 8 C/./for_loop.exe

Hi from thread 2, I am iteration 4
Hi from thread 2, I am iteration 5
Hi from thread 0, I am iteration 0
Hi from thread 0, I am iteration 1
Hi from thread 4, I am iteration 8
Hi from thread 5, I am iteration 9
Hi from thread 1, I am iteration 2
Hi from thread 1, I am iteration 3
Hi from thread 3, I am iteration 6
Hi from thread 3, I am iteration 7


You can play with this code in [C/for_loop.c](C/for_loop.c)

Notice that each thread will sometimes be associated with multiple iterations. An example of an execution with 3 threads could be:

`| T0 | T0 | T1 | T1 | T2 | T2 | T0 | T0 | T1 | T1 |`

In the above `T#` means the thread with `ID = #`.

As we will see later below, it is possible to control assigment of threads to iterations. 

#### A note on variable privatization

A common pitfal is to believe that data privatization happens at the iteration level, rather than at the thread level. The clauses `private`, `firstprivate` and `lastprivate` are used to modify the privatization at the thread level rather than at the iteration level. For this reason, if we consider the thread distribution above, and we use a `private` directive for a variable x, this variable will keep the value for `iterations = 0, 1, 6, 7` in thread T0. 

Take for example the following code:

```C
int x;
int arr[10];
#pragma omp parallel
    #pragma omp for schedule(static, 3) firstprivate(x)
    for (int i = 0; i < 10; i++)
        arr[i] = x++;

for (int i = 0; i < 10; i++)
    printf("arr[%d] = %d\n", i, arr[i]);
```

Ignore for now the schedule clause, it will be introduced briefly, but it helps make this point.

In [141]:
#build
!srun -N 1 -c 8 clang -fopenmp C/for_private_loop.c -o C/for_private_loop.exe

In [142]:
#execute
!srun -N 1 -c 8 C/./for_private_loop.exe

arr[0] = 0
arr[1] = 1
arr[2] = 2
arr[3] = 0
arr[4] = 1
arr[5] = 2
arr[6] = 0
arr[7] = 1
arr[8] = 2
arr[9] = 0


Notice that the variable x is not always 0 for each iteration. Rather, each thread has a private variable x, but the value of x is kept between the iterations performed by that thread.

Play with this code in [C/for_private_loop.C](C/for_private_loop.c).

### Data parallelism

Data parallelism comes from being able to perform the same operation over large amounts of data. This is often expressed in the form of a loop, where the loop iteration variable is used to access elements of an iterable data structure (e.g., `A[i]`). Data parallelism is one of the most common parallelization patterns there exists (also heavily exploited by GPUs and alike devices).

Take the following vector addition example:

```C
#pragma omp parallel for
for (int i = 0; i < 1000; i++)
    c[i] = a[i] + b[i];
```

In this case data parallelism is achieved by distributting the access and computation of the arrays `a`, `b`, and `c` across multiple threads. This is achieved by distributting the iteration space that is directly associated with the variable access.

```NOTE: The `omp parallel for` is a combined directive that is equivalent to writing `parallel` and `for` in two lines. ```

Not every loop can achieve parallelism through a `parallel for` directive. The decision if a loop is or not parallelizable relies on the dependencies between iterations. This concept is first introduce in [Lab1](../Lab1/lab1.ipynb) but let us improve its definition here. 

In the vector addition example, arrays `a` and `b` are read, while array `c` is written. As a developer, we must track these as they could determine if a code is parallelizable. The analysis of inter-loop dependencies is performed by separating reads and writes operations in the code, and determine if a variable that is written in an iteration, is read in another iteration. 

Here is an example of a loop that have interloop dependencies, limitting the available parallelism.

```C
/// a[1] depends on the result of a[0]
for (int i = 1; i < N; i++)
    a[i] = a[i-1] + 1;
```

Here is an example of a code that cannot yet be determined if there are inter loop dependencies:

```C
/// Assuming a is a pointer
for (int i = 0; i < N; i++)
    f(a); // a scapes this scope, the dependency analisys relies on f()
```

In other cases where inter-loop dependencies exists, there are work arounds to achieve parallelism. This is the case of reductions:

```C
/// Reduction for commutative operations
for (int i = 0; i < N; i++) {
    i = i + f();
}
```

Also, it is the case when there is some distance between dependencies in the iteration space that is grater than 1.

```C
/// a[20] depends on a[10], but not on a[11], a[12], ...
for (int i = 10; i < N; i++)
    a[i] = a[i-10] + 1;
```

Likewise, some interloop dependencies can be removed by using a second buffer to store the results:

```C
/// a[1] depends on the previous value in a[2]
for (int i = 1; i < N-1; i++)
    a[i] = a[i+1] + 1;

/// But we can create a second array for the result of this loop
int a[N], b[N];
...
for (int i = 1; i < N-1; i++)
    b[i] = a[i+1] + 1; // Notice that the values of a are now read only
```

### 

## Loop scheduling
Loop scheduling allows us to control the assigment of threads to the iteration space. There are 5 types of scheduling policies in OpenMP:

* Static
* Dynamic
* Guided
* Runtime 
* Auto

Here is a table that summarizes scheduling policies:

| Name    | Chunk Size | Thread Scheduling |
| ------- | ----------| ----------------- |
| Static  | User-defined | Fixed |
| Dynamic | User-defined | Variable |
| Guided  | User-defined Hint | Variable |
| Runtime | User-defined | Determined at runtime |
| Auto    | Determined by compiler | Determined by compiler |


Let us study each of this. 

### Static scheduling
Static scheduling determines that the iteration space is partitioned in chunks of fixed size, and that each chunk is scheduled to a thread statically in round-robin fashion.

Using code in [C/for_loop_static.c](C/for_loop_static.c).

```C
#pragma omp parallel
    #pragma omp for schedule(static:3)
    for (int i = 0; i < 10; i++)
        printf("Hi from thread %d\n, I am iteration %d\n", omp_get_thread_num(), i);
```

In [143]:
#build
!srun -N 1 -c 8 clang -fopenmp C/for_loop_static.c -o C/for_loop_static.exe

In [145]:
#execute
!srun -N 1 -c 8 C/./for_loop_static.exe

Hi from thread 0, I am iteration 0
Hi from thread 0, I am iteration 1
Hi from thread 0, I am iteration 2
Hi from thread 3, I am iteration 9
Hi from thread 2, I am iteration 6
Hi from thread 2, I am iteration 7
Hi from thread 2, I am iteration 8
Hi from thread 1, I am iteration 3
Hi from thread 1, I am iteration 4
Hi from thread 1, I am iteration 5


### Dynamic scheduling
Dynamic scheduling determines that the iteration space is partitioned in chunks of fixed size. However scheduling of each chunk to a thread is dynamically determined at runtime as the threads become free. 

Using code in [C/for_loop_dynamic.c](C/for_loop_dynamic.c).

```C
#pragma omp parallel
    #pragma omp for schedule(dynamic: 3)
    for (int i = 0; i < 10; i++)
        printf("Hi from thread %d\n, I am iteration %d\n", omp_get_thread_num(), i);
```

In [147]:
#build
!srun -N 1 -c 8 clang -fopenmp C/for_loop_dynamic.c -o C/for_loop_dynamic.exe

In [148]:
#execute
!srun -N 1 -c 8 C/./for_loop_dynamic.exe

Hi from thread 5, I am iteration 3
Hi from thread 5, I am iteration 4
Hi from thread 5, I am iteration 5
Hi from thread 4, I am iteration 0
Hi from thread 4, I am iteration 1
Hi from thread 4, I am iteration 2
Hi from thread 0, I am iteration 9
Hi from thread 3, I am iteration 6
Hi from thread 3, I am iteration 7
Hi from thread 3, I am iteration 8


### Guided scheduling
In guided scheduling, the developer provides a hint for the chunk size, but the actual size can be adjusted dynamically at runtime. Likewise, the assignment of chunks can be dynamically scheduled at runtime.

Using code in [C/for_loop_guided.c](C/for_loop_guided.c).

```C
#pragma omp parallel
    #pragma omp for schedule(guided:3)
    for (int i = 0; i < 20; i++)
        printf("Hi from thread %d\n, I am iteration %d\n", omp_get_thread_num(), i);
```

In [150]:
#build
!srun -N 1 -c 8 clang -fopenmp C/for_loop_guided.c -o C/for_loop_guided.exe

In [151]:
#execute
!srun -N 1 -c 8 C/./for_loop_guided.exe

Hi from thread 0, I am iteration 3
Hi from thread 0, I am iteration 4
Hi from thread 0, I am iteration 5
Hi from thread 0, I am iteration 18
Hi from thread 1, I am iteration 12
Hi from thread 1, I am iteration 13
Hi from thread 5, I am iteration 15
Hi from thread 5, I am iteration 16
Hi from thread 5, I am iteration 17
Hi from thread 0, I am iteration 19
Hi from thread 1, I am iteration 14
Hi from thread 3, I am iteration 6
Hi from thread 3, I am iteration 7
Hi from thread 3, I am iteration 8
Hi from thread 4, I am iteration 0
Hi from thread 4, I am iteration 1
Hi from thread 4, I am iteration 2
Hi from thread 2, I am iteration 9
Hi from thread 2, I am iteration 10
Hi from thread 2, I am iteration 11


### Auto scheduling
In Auto scheduling, the scheduling decision is leaft to the compiler. The compiler can use optimizations and heuristics to find the best scheduling option. 

Using code in [C/for_loop_auto.c](C/for_loop_auto.c).

```C
#pragma omp parallel
    #pragma omp for schedule(auto)
    for (int i = 0; i < 10; i++)
        printf("Hi from thread %d\n, I am iteration %d\n", omp_get_thread_num(), i);
```

In [152]:
#build
!srun -N 1 -c 8 clang -fopenmp C/for_loop_auto.c -o C/for_loop_auto.exe

In [153]:
#execute
!srun -N 1 -c 8 C/./for_loop_auto.exe

Hi from thread 0, I am iteration 1
Hi from thread 0, I am iteration 6
Hi from thread 0, I am iteration 7
Hi from thread 0, I am iteration 8
Hi from thread 3, I am iteration 2
Hi from thread 5, I am iteration 5
Hi from thread 4, I am iteration 0
Hi from thread 1, I am iteration 4
Hi from thread 0, I am iteration 9
Hi from thread 2, I am iteration 3


### Runtime scheduling
The runtime scheduling policy allows to make changes to the scheduling at runtime, by using an environment variable

Using code in [C/for_loop_runtime.c](C/for_loop_runtime.c).

```C
#pragma omp parallel
    #pragma omp for schedule(runtime)
    for (int i = 0; i < 10; i++)
        printf("Hi from thread %d\n, I am iteration %d\n", omp_get_thread_num(), i);
```

In [154]:
#build
!srun -N 1 -c 8 clang -fopenmp C/for_loop_runtime.c -o C/for_loop_runtime.exe

In [156]:
#execute
!srun -N 1 -c 8 C/./for_loop_runtime.exe

Hi from thread 1, I am iteration 2
Hi from thread 1, I am iteration 3
Hi from thread 2, I am iteration 4
Hi from thread 2, I am iteration 5
Hi from thread 4, I am iteration 8
Hi from thread 0, I am iteration 0
Hi from thread 0, I am iteration 1
Hi from thread 5, I am iteration 9
Hi from thread 3, I am iteration 6
Hi from thread 3, I am iteration 7


In [159]:
#execute
!OMP_SCHEDULE="static,5" srun -N 1 -c 8 C/./for_loop_runtime.exe

Hi from thread 1, I am iteration 5
Hi from thread 1, I am iteration 6
Hi from thread 1, I am iteration 7
Hi from thread 1, I am iteration 8
Hi from thread 1, I am iteration 9
Hi from thread 0, I am iteration 0
Hi from thread 0, I am iteration 1
Hi from thread 0, I am iteration 2
Hi from thread 0, I am iteration 3
Hi from thread 0, I am iteration 4


In [160]:
#execute
!OMP_SCHEDULE="dynamic,2" srun -N 1 -c 8  C/./for_loop_runtime.exe

Hi from thread 3, I am iteration 2
Hi from thread 3, I am iteration 3
Hi from thread 0, I am iteration 4
Hi from thread 0, I am iteration 5
Hi from thread 1, I am iteration 8
Hi from thread 1, I am iteration 9
Hi from thread 2, I am iteration 6
Hi from thread 2, I am iteration 7
Hi from thread 4, I am iteration 0
Hi from thread 4, I am iteration 1


### Excercise - Matrix multiplication

Parallelize the following Matrix Multiplication algorithm. Modify the file [Exercises/matrix_mult.c](Exercises/matrix_mult.c).

```C
for (int i = 0; i < N; i++)
{
    for (int j = 0; j < N; j++)
    {
        *(C + i * N + j) = 0;
        for (int k = 0; k < N; k++)
        {
            *(C + i * N + j) += *(A + i * N + k) * *(B + k * N + j);
        }
    }
}
```


In [171]:
#build
!srun -N 1 -c 8 clang -fopenmp Exercises/matrix_mult.c -o Exercises/matrix_mult.exe

In [172]:
#execute
!srun -N 1 -c 8 Exercises/./matrix_mult.exe

285 240 195 150 105 60 15 -30 -75 -120 
330 275 220 165 110 55 0 -55 -110 -165 
375 310 245 180 115 50 -15 -80 -145 -210 
420 345 270 195 120 45 -30 -105 -180 -255 
465 380 295 210 125 40 -45 -130 -215 -300 
510 415 320 225 130 35 -60 -155 -250 -345 
555 450 345 240 135 30 -75 -180 -285 -390 
600 485 370 255 140 25 -90 -205 -320 -435 
645 520 395 270 145 20 -105 -230 -355 -480 
690 555 420 285 150 15 -120 -255 -390 -525 


Extra points: Changing the value of N. Can you create a plot for speed up of your algorithm?

Here is a simple solution in the file [Solutions/Matrix_mult.c](Solutions/matrix_mult.c)

In [214]:
#build
!srun -N 1 -c 8 gcc -fopenmp Solutions/matrix_mult.c -o Solutions/matrix_mult.exe -O3

In [215]:
#execute
!srun -N 1 -c 8 Solutions/./matrix_mult.exe

Time spent in sequential: 1.925665
Time spent in parallel: 1.915171
